#### Imports

In [2]:
import json
import nltk
from nltk.collocations import *
import datetime as dt
import locale
import spacy
from tqdm import tqdm
import pprint
import pandas as pd
from ast import literal_eval
import os
import gensim
import gensim.corpora as corpora
from pprint import pprint
from tqdm import tqdm 


In [ ]:
!python -m spacy download de_core_news_lg

In [ ]:
#NOTE: default version doesntwork in colab
!pip install pyLDAvis==2.1.2

#### Read Data

In [6]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#change cwd
%cd drive/MyDrive/Work/Frontline/data
#%cd /content/drive/MyDrive/data/

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [8]:
# paths
FILTERED_PATH="filtered_4_26"

In [9]:
dfs = []

# loop through files 
for filename in tqdm(os.listdir(FILTERED_PATH)):
    # if csv file, load and add to dfs  
    if filename.endswith(".csv"):
        file_path = os.path.join(FILTERED_PATH, filename)
        df = pd.read_csv(file_path, index_col=0, converters={"text":literal_eval})
        dfs.append(df)

# combine files in df
df = pd.concat(dfs, ignore_index=True)

100%|██████████| 207/207 [00:14<00:00, 14.08it/s]


### Topic Modelling


#### Prepare Data


In [10]:
# custom module
import preprocessing

In [11]:
# Load model
spacy_mod = spacy.load("de_core_news_lg", disable=['ner', 'parser', 'tagger'])

In [12]:
# read custom stopwords

# open list of custom stopwords
custom_stop_words= open("custom_stopwords.txt").read().split()

# add custom stopwords to model
for word in custom_stop_words:
  spacy_mod.Defaults.stop_words.add(word)

In [13]:
smaller_df=df.iloc[:10000,:]

In [14]:
# convert corpus to language object
spacy_lang = []
for i, doc in tqdm(smaller_df.iterrows()): 
  spacy_lang.append(spacy_mod("".join(doc['text'])))

10000it [05:28, 30.48it/s]


In [15]:
# preprocess: remove stopwords
spacy_cleaned = []
for doc in tqdm(spacy_lang): 
    spacy_cleaned.append(preprocessing.preprocess(doc, remove_ent=True))

100%|██████████| 10000/10000 [00:04<00:00, 2029.38it/s]


#### Topic Analysis

In [16]:
import pyLDAvis
import pyLDAvis.gensim
import pickle  

#### Creating the model

In [36]:
# Create Dictionary
id2word = corpora.Dictionary(spacy_cleaned)

# Create Corpus
texts = spacy_cleaned

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word, num_topics=num_topics)
# Print the key words for each topic
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [38]:
# saving model
lda_model.save("models/tmodel_larger_ds")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Loading a previously saved model

In [ ]:
lda_model=gensim.models.LdaMulticore.load("models/tmodel_larger_ds", mmap="r")
lda_model.print_topics()

In [41]:
# Create Corpus
texts = spacy_cleaned

# Saving dictionary of imported model
id2word=lda_model.id2word

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Vizualizing models

In [42]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.003702  0.073283       1        1  16.023027
4      0.042404 -0.017120       2        1  12.545557
1      0.005822 -0.014541       3        1  11.883438
2      0.083091 -0.048266       4        1  11.689988
7      0.005181  0.032303       5        1  11.639781
0     -0.136787 -0.034374       6        1  10.583703
9      0.045987  0.023286       7        1  10.188382
8     -0.019300  0.003206       8        1   6.676984
5     -0.041608  0.007430       9        1   5.987085
6      0.018912 -0.025207      10        1   2.782054, topic_info=                      Term          Freq         Total Category  logprob  \
152                telefon  20786.000000  20786.000000  Default  30.0000   
46     bereitschaftsdienst   9347.000000   9347.000000  Default  29.0000   
2394                    do   4237.000000   4237.000000  Default  28.0000   
2396                    mi   3960.000000   3960.000000  Default  27.0000   
13301               kummer   2578.000000   2578.000000  Default  26.0000   
...                    ...           ...           ...      ...      ...   
352                prozent     65.958642   2990.241568  Topic10  -6.1767   
539                   mann     64.470524   2883.782094  Topic10  -6.1995   
120              notdienst     59.178489   2303.684880  Topic10  -6.2852   
31                   allee     57.771302   2023.322506  Topic10  -6.3093   
286                  fälle     58.274864   2377.108569  Topic10  -6.3006   

       loglift  
152    30.0000  
46     29.0000  
2394   28.0000  
2396   27.0000  
13301  26.0000  
...        ...  
352    -0.2321  
539    -0.2187  
120    -0.0797  
31      0.0260  
286    -0.1265  

[868 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
1445       1  0.359630             abend
1445       2  0.081207             abend
1445       3  0.111369             abend
1445       4  0.081207             abend
1445       5  0.081207             abend
...      ...       ...               ...
52877      5  0.692908     überflutungen
8443       1  0.116438       überlebende
8443       2  0.582188       überlebende
8443       3  0.116438       überlebende
52878      5  0.737388  überschwemmungen

[2950 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 3, 8, 1, 10, 9, 6, 7])

END OF CODE